In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [4]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312      1.0  76.716477                  NaN        NaN
0.000312      NaN        NaN                  1.0  38.907706
0.000750      2.0  40.975266                  2.0  33.904352
0.001052      3.0  33.041806                  3.0  31.687024
0.001476      4.0  26.177273                  4.0  28.647704
0.002070      5.0  20.453729                  5.0  24.715578
0.002904      6.0  15.887172                  6.0  20.074775
0.004074      7.0  12.355622                  7.0  15.169384
0.005714      8.0   9.670589                  8.0  10.650433
0.008015      9.0   7.632345                  9.0   7.176501
0.011243     10.0   6.059436                 10.0   5.028832
0.015771     11.0   4.801153                 11.0   3.957223
0.022122     12.0   3.733531                 12.0   3.369512
0.031031     13.0   2.741821                 13.0   2.795973
0.043528     14.0   1.535444                 14.0   1.821742
0.061057     15.0   0.621692                 15.0   0.940916
0.085645     16.0   1.009796                 16.0   1.170222
0.120136     17.0   1.979216                 17.0   1.897737
0.168516     18.0   2.674290                 18.0   2.340740
0.236378     19.0   3.354145                 19.0   2.850986
0.331549     20.0   4.138402                 20.0   3.733086
0.465100     21.0   5.134848                 21.0   5.080730
0.652400     22.0   6.676295                 22.0   6.966495
0.915100     23.0   8.422255                 23.0   8.670709
1.283650     24.0   8.121785                 24.0   7.954511
1.800600     25.0   6.305420                 25.0   5.861247
2.525700     26.0   4.920157                 26.0   4.725444
3.542800     27.0   3.938123                 27.0   4.202704
4.969550     28.0   3.204642                 28.0   3.479371
6.970850     29.0   2.670490                 29.0   2.690186
9.778100     30.0   2.245281                 30.0   2.197952
13.715850    31.0   1.823620                 31.0   2.034601
19.239350    32.0   1.385848                 32.0   1.708755
26.987250    33.0   1.051379                 33.0   1.212175
37.855300    34.0   0.824175                 34.0   0.818724
53.100050    35.0   0.576412                 35.0   0.620058
73.887500    36.0   0.318304                 36.0   0.429622
97.662500    37.0   0.177325                 37.0   0.244621
121.437500   38.0   0.132179                 38.0   0.136684
145.212500   39.0   0.033080                 39.0  -0.004373
168.987500   40.0  -0.099825                 40.0  -0.158236
192.762500   41.0  -0.095180                 41.0  -0.144768
216.537500   42.0   0.024981                 42.0  -0.001751
240.312500   43.0   0.077872                 43.0   0.058542
264.087500   44.0   0.094879                 44.0   0.075069
287.862500   45.0   0.101746                 45.0   0.082631
311.637500   46.0   0.103508                 46.0   0.087294
335.412500   47.0   0.102678                 47.0   0.090824
359.187500   48.0   0.100513                 48.0   0.092659
382.962500   49.0   0.097735                 49.0   0.093209
406.737500   50.0   0.094659                 50.0   0.092349
430.512500   51.0   0.090941                 51.0   0.089818
454.287500   52.0   0.085924                 52.0   0.084335
478.062500   53.0   0.079580                 53.0   0.077044
501.837500   54.0   0.072646                 54.0   0.068422
525.612500   55.0   0.066034                 55.0   0.060128
549.387500   56.0   0.060212                 56.0   0.052434
573.162500   57.0   0.055206                 57.0   0.045523
596.937500   58.0   0.051038                 58.0   0.039582
620.712500   59.0   0.047189                 59.0   0.034311
644.487500   60.0   0.043572                 60.0   0.030018
668.262500   61.0   0.040239                 61.0   0.026349
692.037500   62.0   0.037272                 62.0

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -17.738220   0.000000 -17.738220        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -17.435557   0.000182   
6.244000e-04 2     -17.738562   0.006016 -17.732546 -17.435921   0.003424   
8.759000e-04 3     -17.738717   0.007392 -17.731325 -17.436070   0.004583   
1.228600e-03 4     -17.738915   0.008970 -17.729945 -17.436264   0.006101   
1.723400e-03 5     -17.739163   0.010752 -17.728410 -17.436517   0.008034   
2.417400e-03 6     -17.739476   0.012748 -17.726729 -17.436848   0.010396   
3.390900e-03 7     -17.739876   0.014979 -17.724897 -17.437280   0.013144   
4.756500e-03 8     -17.740389   0.017491 -17.722898 -17.437849   0.016167   
6.672000e-03 9     -17.741055   0.020352 -17.720704 -17.438603   0.019338   
9.358900e-03 10    -17.741933   0.023659 -17.718275 -17.439608   0.022627   
1.312780e-02 11    -17.743108   0.027539 -17.715569 -17.440967   0.026232   
1.841450e-02 12    -17.744713   0.032150 -17.712563 -17.442840   0.030583   
2.583020e-02 13    -17.746962   0.037679 -17.709283 -17.445471   0.036175   
3.623230e-02 14    -17.750245   0.044340 -17.705904 -17.449279   0.043428   
5.082340e-02 15    -17.755665   0.052414 -17.703251 -17.455368   0.052666   
7.129060e-02 16    -17.764995   0.063252 -17.701743 -17.465431   0.065011   
1.000000e-01 17    -17.778772   0.080463 -17.698309 -17.479946   0.083506   
1.402710e-01 18    -17.796685   0.107817 -17.688868 -17.498614   0.111227   
1.967600e-01 19    -17.819499   0.148526 -17.670972 -17.522119   0.150398   
2.759970e-01 20    -17.847890   0.208400 -17.639490 -17.550971   0.206012   
3.871000e-01 21    -17.881570   0.296545 -17.585025 -17.584547   0.288724   
5.431000e-01 22    -17.917160   0.427024 -17.490136 -17.618913   0.416987   
7.617000e-01 23    -17.937519   0.620264 -17.317255 -17.636730   0.615219   
1.068500e+00 24    -17.900548   0.889380 -17.011168 -17.599147   0.892785   
1.498800e+00 25    -17.793142   1.195958 -16.597184 -17.495019   1.194157   
2.102400e+00 26    -17.646393   1.500051 -16.146342 -17.352111   1.470376   
2.949000e+00 27    -17.470616   1.817696 -15.652920 -17.179674   1.771883   
4.136600e+00 28    -17.267239   2.168333 -15.098906 -16.977586   2.161091   
5.802500e+00 29    -17.038372   2.571863 -14.466509 -16.748169   2.618356   
8.139200e+00 30    -16.780484   3.053163 -13.727321 -16.483861   3.098766   
1.141700e+01 31    -16.484836   3.629309 -12.855526 -16.174357   3.642768   
1.601470e+01 32    -16.157769   4.295441 -11.862327 -15.825794   4.402425   
2.246400e+01 33    -15.841878   5.038291 -10.803587 -15.486239   5.368435   
3.151050e+01 34    -15.559863   5.882957  -9.676906 -15.175999   6.357324   
4.420010e+01 35    -15.278108   6.840081  -8.438027 -14.856593   7.268728   
6.200000e+01 36    -15.025457   7.802808  -7.222649 -14.558209   8.277888   
8.577500e+01 37    -14.952916   8.626712  -6.326204 -14.445288   9.375045   
1.095500e+02 38    -15.053452   9.226652  -5.826801 -14.510187  10.128945   
1.333250e+02 39    -15.216914   9.762371  -5.454543 -14.633158  10.636903   
1.571000e+02 40    -15.556524  10.195144  -5.361380 -14.945140  10.936568   
1.808750e+02 41    -16.302836  10.660318  -5.642518 -15.708628  11.254366   
2.046500e+02 42    -17.388686  11.478110  -5.910576 -16.845845  11.983828   
2.284250e+02 43    -18.534089  12.693867  -5.840222 -18.044500  13.177551   
2.522000e+02 44    -19.666977  14.046066  -5.620911 -19.223467  14.521409   
2.759750e+02 45    -20.782041  15.428340  -5.353700 -20.379300  15.888682   
2.997500e+02 46    -21.878752  16.811600  -5.067153 -21.512861  17.254983   
3.235250e+02 47    -22.955992  18.180351  -4.775642 -22.621796  18.609791   
3.473000e+02 48    -24.012674  19.526206  -4.486468 -23.707322  19.951132   
3.710750e+02 49    -25

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')